In [2]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [4]:
from sklearn.base import BaseEstimator, clone
from sklearn.model_selection import train_test_split
import copy
import pandas as pd
import numpy as np

from dask.base import is_dask_collection
import dask.dataframe as dd

In [5]:
from utils import config

In [6]:
from sklearn.model_selection import StratifiedKFold, KFold
from utils import dasker, logging, config
from dask.dataframe import from_pandas
import xgboost as xgb
import dask.array as da
import dask.distributed



In [7]:
from utils import dasker, config as util_config

In [8]:

client = dasker.get_dask_client()


# X and y must be Dask dataframes or arrays
num_obs = 1e5
num_features = 20
X = da.random.random(size=(num_obs, num_features), chunks=(1000, num_features))
y = da.random.random(size=(num_obs, 1), chunks=(1000, 1))

dtrain = xgb.dask.DaskDMatrix(client, X, y)

output = xgb.dask.train(
    client,
    {"verbosity": 2, "tree_method": "hist", "objective": "reg:squarederror"},
    dtrain,
    num_boost_round=4,
    evals=[(dtrain, "train")],
)


2022-09-07 17:09:50,768 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vlz45mj9', purging
2022-09-07 17:09:50,769 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-cecv0jk5', purging
2022-09-07 17:09:50,770 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-gxvoh7zh', purging
2022-09-07 17:09:50,771 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wph58ycw', purging
/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/dask.py:855: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[17:09:52] task [xgboost.dask-0]:tcp://127.0.0.1:40007 got new rank 0
[17:09:52] task [xgboost.dask-1]:tcp://127.0.0.1:44045 got new rank 1
[17:09:52] task [xgboost.dask-2]:tcp://127.0.0.1:43763 got new rank 2
[17:09:52] task [xgboost.d

[0]	train-rmse:0.28911
[1]	train-rmse:0.28879
[2]	train-rmse:0.28847
[3]	train-rmse:0.28815


[17:09:52] INFO: /mnt/c/Users/rwmas/xgboost/src/gbm/gbtree.cc:178: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[17:09:52] INFO: /mnt/c/Users/rwmas/xgboost/src/gbm/gbtree.cc:178: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[17:09:52] INFO: /mnt/c/Users/rwmas/xgboost/src/gbm/gbtree.cc:178: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[17:09:52] INFO: /mnt/c/Users/rwmas/xgboost/src/gbm/gbtree.cc:178: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[17:09:52] INFO: /mnt/c/Users/rwmas/xgboost/src/gbm/gbtree.cc:178: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[17:09:52] INFO: /mnt/c/Users/rwmas/xgboost/src/gbm/gbtree.cc:178: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.[17:09:52] INFO: /mnt/c/Users/rwmas/xgboost/src/gbm/g

In [46]:
from sklearn.metrics import mean_squared_error

result_tain_list = []
result_test_list = []


param = {"verbosity": 0, "tree_method": "hist", "objective": "reg:squarederror",
                         "max_depth": "10", "learning_rate": 0.01, "max_bin": 256 }


for train_idx, test_idx in cv.split(X_base, y_base):
    X_train, y_train = X_base[train_idx], y_base[train_idx]
    X_test, y_test = X_base[test_idx], y_base[test_idx]
    dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)
    dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)

    best_model = xgb.dask.train(
        client,
        param, 
        dtrain,
        num_boost_round=500,
        # early_stopping_rounds=50, 
        evals=[(dtrain, "train")],
    )
    
    # booster = output["booster"]
    # print(booster.best_iteration)
    # best_model = booster[: booster.best_iteration]
    train_preds = xgb.dask.predict(client, best_model, dtrain)
    test_preds = xgb.dask.predict(client, best_model, dtest)

    result_tain_list.append(mean_squared_error(train_preds, y_train))
    result_test_list.append(mean_squared_error(test_preds, y_test))




/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/dask.py:855: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[11:11:02] task [xgboost.dask-0]:tcp://127.0.0.1:40007 got new rank 0
[11:11:02] task [xgboost.dask-1]:tcp://127.0.0.1:44045 got new rank 1
[11:11:02] task [xgboost.dask-2]:tcp://127.0.0.1:43763 got new rank 2
[11:11:02] task [xgboost.dask-3]:tcp://127.0.0.1:38021 got new rank 3


[0]	train-rmse:0.28931
[1]	train-rmse:0.28920
[2]	train-rmse:0.28907
[3]	train-rmse:0.28897
[4]	train-rmse:0.28885
[5]	train-rmse:0.28873
[6]	train-rmse:0.28862
[7]	train-rmse:0.28851
[8]	train-rmse:0.28838
[9]	train-rmse:0.28828
[10]	train-rmse:0.28817
[11]	train-rmse:0.28808
[12]	train-rmse:0.28794
[13]	train-rmse:0.28784
[14]	train-rmse:0.28772
[15]	train-rmse:0.28761
[16]	train-rmse:0.28751
[17]	train-rmse:0.28739
[18]	train-rmse:0.28729
[19]	train-rmse:0.28718
[20]	train-rmse:0.28710
[21]	train-rmse:0.28699
[22]	train-rmse:0.28690
[23]	train-rmse:0.28682
[24]	train-rmse:0.28671
[25]	train-rmse:0.28661
[26]	train-rmse:0.28652
[27]	train-rmse:0.28642
[28]	train-rmse:0.28633
[29]	train-rmse:0.28623
[30]	train-rmse:0.28611
[31]	train-rmse:0.28603
[32]	train-rmse:0.28594
[33]	train-rmse:0.28582
[34]	train-rmse:0.28571
[35]	train-rmse:0.28562
[36]	train-rmse:0.28551
[37]	train-rmse:0.28544
[38]	train-rmse:0.28534
[39]	train-rmse:0.28524
[40]	train-rmse:0.28515
[41]	train-rmse:0.28506
[4

/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/dask.py:855: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[11:12:33] task [xgboost.dask-0]:tcp://127.0.0.1:40007 got new rank 0
[11:12:33] task [xgboost.dask-1]:tcp://127.0.0.1:44045 got new rank 1
[11:12:33] task [xgboost.dask-3]:tcp://127.0.0.1:38021 got new rank 2


[0]	train-rmse:0.28956
[1]	train-rmse:0.28944
[2]	train-rmse:0.28933
[3]	train-rmse:0.28919
[4]	train-rmse:0.28908
[5]	train-rmse:0.28898
[6]	train-rmse:0.28884
[7]	train-rmse:0.28873
[8]	train-rmse:0.28860
[9]	train-rmse:0.28849
[10]	train-rmse:0.28836
[11]	train-rmse:0.28826
[12]	train-rmse:0.28815
[13]	train-rmse:0.28804
[14]	train-rmse:0.28793
[15]	train-rmse:0.28781
[16]	train-rmse:0.28770
[17]	train-rmse:0.28758
[18]	train-rmse:0.28747
[19]	train-rmse:0.28736
[20]	train-rmse:0.28727
[21]	train-rmse:0.28714
[22]	train-rmse:0.28704
[23]	train-rmse:0.28694
[24]	train-rmse:0.28683
[25]	train-rmse:0.28672
[26]	train-rmse:0.28661
[27]	train-rmse:0.28649
[28]	train-rmse:0.28641
[29]	train-rmse:0.28630
[30]	train-rmse:0.28621
[31]	train-rmse:0.28611
[32]	train-rmse:0.28602
[33]	train-rmse:0.28592
[34]	train-rmse:0.28583
[35]	train-rmse:0.28574
[36]	train-rmse:0.28563
[37]	train-rmse:0.28553
[38]	train-rmse:0.28543
[39]	train-rmse:0.28534
[40]	train-rmse:0.28525
[41]	train-rmse:0.28516
[4

/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/dask.py:855: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[11:13:36] task [xgboost.dask-0]:tcp://127.0.0.1:40007 got new rank 0
[11:13:36] task [xgboost.dask-1]:tcp://127.0.0.1:44045 got new rank 1
[11:13:36] task [xgboost.dask-2]:tcp://127.0.0.1:43763 got new rank 2
[11:13:36] task [xgboost.dask-3]:tcp://127.0.0.1:38021 got new rank 3


[0]	train-rmse:0.28986
[1]	train-rmse:0.28973
[2]	train-rmse:0.28960
[3]	train-rmse:0.28948
[4]	train-rmse:0.28935
[5]	train-rmse:0.28921
[6]	train-rmse:0.28908
[7]	train-rmse:0.28895
[8]	train-rmse:0.28883
[9]	train-rmse:0.28869
[10]	train-rmse:0.28856
[11]	train-rmse:0.28845
[12]	train-rmse:0.28835
[13]	train-rmse:0.28821
[14]	train-rmse:0.28808
[15]	train-rmse:0.28797
[16]	train-rmse:0.28784
[17]	train-rmse:0.28774
[18]	train-rmse:0.28761
[19]	train-rmse:0.28749
[20]	train-rmse:0.28737
[21]	train-rmse:0.28724
[22]	train-rmse:0.28714
[23]	train-rmse:0.28702
[24]	train-rmse:0.28694
[25]	train-rmse:0.28681
[26]	train-rmse:0.28667
[27]	train-rmse:0.28654
[28]	train-rmse:0.28648
[29]	train-rmse:0.28634
[30]	train-rmse:0.28625
[31]	train-rmse:0.28613
[32]	train-rmse:0.28607
[33]	train-rmse:0.28596
[34]	train-rmse:0.28581
[35]	train-rmse:0.28568
[36]	train-rmse:0.28554
[37]	train-rmse:0.28546
[38]	train-rmse:0.28532
[39]	train-rmse:0.28525
[40]	train-rmse:0.28517
[41]	train-rmse:0.28504
[4

In [47]:

holdout_pred = xgb.dask.predict(client, best_model, X_holdout)

train_test_diff = np.abs(np.mean(result_tain_list) - np.mean(result_test_list))       

print('MEAN TRAIN.....', np.mean(result_tain_list))
print('MEAN TEST.....', np.mean(result_test_list))
print('TRAIN-TEST DIFF.....', train_test_diff)
print('HOLDOUT ..... ', mean_squared_error(holdout_pred, y_holdout))


MEAN TRAIN..... 0.060298736358880174
MEAN TEST..... 0.08494893198384584
TRAIN-TEST DIFF..... 0.02465019562496567
HOLDOUT .....  0.0830387511983989


In [55]:
from sklearn.model_selection import StratifiedKFold, KFold



client = dasker.get_dask_client()
print(f'Local dask client created: localhost:{util_config.dask_local_scheduler_port}')

cv = KFold(n_splits=3, random_state=None, shuffle=False)

briskmodels = BriskXGBoost(client, cv, logger = None, pred_type = 'regression')
res = briskmodels.fit(X,y )


Local dask client created: localhost:8787


/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/dask.py:855: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[11:57:35] task [xgboost.dask-0]:tcp://127.0.0.1:40007 got new rank 0
[11:57:35] task [xgboost.dask-1]:tcp://127.0.0.1:44045 got new rank 1
[11:57:35] task [xgboost.dask-2]:tcp://127.0.0.1:43763 got new rank 2
[11:57:35] task [xgboost.dask-3]:tcp://127.0.0.1:38021 got new rank 3


[0]	train-rmse:0.28932
[1]	train-rmse:0.28919
[2]	train-rmse:0.28907
[3]	train-rmse:0.28896
[4]	train-rmse:0.28885
[5]	train-rmse:0.28873
[6]	train-rmse:0.28862
[7]	train-rmse:0.28852
[8]	train-rmse:0.28843
[9]	train-rmse:0.28831
[10]	train-rmse:0.28821
[11]	train-rmse:0.28811
[12]	train-rmse:0.28800
[13]	train-rmse:0.28790
[14]	train-rmse:0.28777
[15]	train-rmse:0.28767
[16]	train-rmse:0.28757
[17]	train-rmse:0.28748
[18]	train-rmse:0.28737
[19]	train-rmse:0.28728
[20]	train-rmse:0.28718
[21]	train-rmse:0.28710
[22]	train-rmse:0.28700
[23]	train-rmse:0.28690
[24]	train-rmse:0.28678
[25]	train-rmse:0.28669
[26]	train-rmse:0.28659
[27]	train-rmse:0.28647
[28]	train-rmse:0.28637
[29]	train-rmse:0.28625
[30]	train-rmse:0.28617
[31]	train-rmse:0.28606
[32]	train-rmse:0.28595
[33]	train-rmse:0.28585
[34]	train-rmse:0.28574
[35]	train-rmse:0.28563
[36]	train-rmse:0.28555
[37]	train-rmse:0.28543
[38]	train-rmse:0.28534
[39]	train-rmse:0.28527
[40]	train-rmse:0.28518
[41]	train-rmse:0.28511
[4

/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/dask.py:855: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[11:58:00] task [xgboost.dask-0]:tcp://127.0.0.1:40007 got new rank 0
[11:58:00] task [xgboost.dask-1]:tcp://127.0.0.1:44045 got new rank 1
[11:58:00] task [xgboost.dask-2]:tcp://127.0.0.1:43763 got new rank 2
[11:58:00] task [xgboost.dask-3]:tcp://127.0.0.1:38021 got new rank 3


[0]	train-rmse:0.28956
[1]	train-rmse:0.28945
[2]	train-rmse:0.28934
[3]	train-rmse:0.28924
[4]	train-rmse:0.28914
[5]	train-rmse:0.28903
[6]	train-rmse:0.28893
[7]	train-rmse:0.28882
[8]	train-rmse:0.28872
[9]	train-rmse:0.28863
[10]	train-rmse:0.28853
[11]	train-rmse:0.28843
[12]	train-rmse:0.28833
[13]	train-rmse:0.28823
[14]	train-rmse:0.28809
[15]	train-rmse:0.28800
[16]	train-rmse:0.28787
[17]	train-rmse:0.28775
[18]	train-rmse:0.28765
[19]	train-rmse:0.28752
[20]	train-rmse:0.28743
[21]	train-rmse:0.28732
[22]	train-rmse:0.28723
[23]	train-rmse:0.28709
[24]	train-rmse:0.28699
[25]	train-rmse:0.28690
[26]	train-rmse:0.28681
[27]	train-rmse:0.28673
[28]	train-rmse:0.28658
[29]	train-rmse:0.28650
[30]	train-rmse:0.28639
[31]	train-rmse:0.28629
[32]	train-rmse:0.28615
[33]	train-rmse:0.28606
[34]	train-rmse:0.28594
[35]	train-rmse:0.28585
[36]	train-rmse:0.28577
[37]	train-rmse:0.28563
[38]	train-rmse:0.28553
[39]	train-rmse:0.28544
[40]	train-rmse:0.28536
[41]	train-rmse:0.28528
[4

/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/dask.py:855: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[11:58:21] task [xgboost.dask-0]:tcp://127.0.0.1:40007 got new rank 0
[11:58:21] task [xgboost.dask-1]:tcp://127.0.0.1:44045 got new rank 1
[11:58:21] task [xgboost.dask-2]:tcp://127.0.0.1:43763 got new rank 2
[11:58:21] task [xgboost.dask-3]:tcp://127.0.0.1:38021 got new rank 3


[0]	train-rmse:0.28985
[1]	train-rmse:0.28971
[2]	train-rmse:0.28958
[3]	train-rmse:0.28945
[4]	train-rmse:0.28932
[5]	train-rmse:0.28919
[6]	train-rmse:0.28905
[7]	train-rmse:0.28891
[8]	train-rmse:0.28877
[9]	train-rmse:0.28865
[10]	train-rmse:0.28855
[11]	train-rmse:0.28841
[12]	train-rmse:0.28831
[13]	train-rmse:0.28818
[14]	train-rmse:0.28808
[15]	train-rmse:0.28800
[16]	train-rmse:0.28786
[17]	train-rmse:0.28775
[18]	train-rmse:0.28766
[19]	train-rmse:0.28752
[20]	train-rmse:0.28740
[21]	train-rmse:0.28732
[22]	train-rmse:0.28720
[23]	train-rmse:0.28713
[24]	train-rmse:0.28702
[25]	train-rmse:0.28690
[26]	train-rmse:0.28681
[27]	train-rmse:0.28669
[28]	train-rmse:0.28661
[29]	train-rmse:0.28647
[30]	train-rmse:0.28635
[31]	train-rmse:0.28627
[32]	train-rmse:0.28618
[33]	train-rmse:0.28605
[34]	train-rmse:0.28597
[35]	train-rmse:0.28585
[36]	train-rmse:0.28578
[37]	train-rmse:0.28564
[38]	train-rmse:0.28551
[39]	train-rmse:0.28538
[40]	train-rmse:0.28524
[41]	train-rmse:0.28513
[4

In [56]:
res

{'mean_train_err': 0.07813503569086387,
 'mean_test_err': 0.0841429247638892,
 'train_test_err_diff': 0.006007889073025324,
 'holdout_err': 0.08260488788395538}

100

In [ ]:
param = {"verbosity": 0, "tree_method": "hist", "objective": "reg:squarederror",
                         "max_depth": "10", "learning_rate": 0.01, "max_bin": 256, "num_boost_round" : 100}

In [57]:
from dask_ml.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


class BriskXGBoost():
    def __init__(self, client, cv, logger, pred_type) -> None:
        self.client = client 
        self.cv = cv
        self.logger = logger
        self.mean_train_error = None
        self.mean_test_error = None
        self.holdout_error = None
        self.pred_type = pred_type
        self.model = None
        

    def fit(self, X, y):

        result_tain_list = []
        result_test_list = []

        X_base, X_holdout, y_base, y_holdout = train_test_split(X, y, 
                                                                                                                                    random_state=config.rand_state)

        for train_idx, test_idx in cv.split(X_base, y_base):
            X_train, y_train = X_base[train_idx], y_base[train_idx]
            X_test, y_test = X_base[test_idx], y_base[test_idx]
            dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)
            dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)

            trained_model = xgb.dask.train(
                client,
                param, 
                dtrain,
                num_boost_round=param["num_boost_round"],
                # early_stopping_rounds=50, 
                evals=[(dtrain, "train")],
            )
            
            # booster = output["booster"]
            # print(booster.best_iteration)
            # best_model = booster[: booster.best_iteration]
            train_preds = xgb.dask.predict(client, trained_model, dtrain)
            test_preds = xgb.dask.predict(client, trained_model, dtest)

            if self.pred_type == 'classification': 
                result_tain_list = -1
                result_test_list = -1
            else:
                result_tain_list.append(mean_squared_error(train_preds, y_train))
                result_test_list.append(mean_squared_error(test_preds, y_test))

        
        
        self.mean_train_err = np.mean(result_tain_list)
        self.mean_test_err = np.mean(result_test_list)
        self.train_test_err_diff = np.abs(self.mean_train_err - self.mean_test_err)


        holdout_pred = xgb.dask.predict(client, trained_model, X_holdout)

        if self.pred_type == 'classification': 
            self.holdout_err = -1
        else:
            self.holdout_err = mean_squared_error(holdout_pred, y_holdout)    


        self.model = trained_model

        return {
            "mean_train_err": self.mean_train_err,
            "mean_test_err": self.mean_test_err,
            "train_test_err_diff": self.train_test_err_diff,
            "holdout_err": self.holdout_err
        }


    def get_result_stats(self):

        return {
            "mean_train_err": self.mean_train_err,
            "mean_test_err": self.mean_test_err,
            "train_test_err_diff": self.train_test_err_diff,
            "holdout_err": self.holdout_err
        }



99


In [37]:
best_model

In [38]:
prediction = xgb.dask.predict(client, best_model, dtrain)

In [39]:
prediction

dask.array<concatenate, shape=(30000,), dtype=float32, chunksize=(900,), chunktype=numpy.ndarray>

In [ ]:



X_base, X_holdout, y_base, y_holdout = train_test_split(
    X, y, random_state=config.rand_state)

split_scores_ = []
holdout_scores_ = []
estimators_ = []            



for train_idx, test_idx in cv.split(X_base, y_base):
    

    # print(test_idx)

    X_test, y_test = X_base[test_idx], y_base[test_idx]
    X_train, y_train = X_base.loc[X_base.index[train_idx]], y_base.loc[y_base.index[train_idx]]


    # X_test, y_test = X_base[test_idx], y_base[test_idx]
    # X_train, y_train = X_base[train_idx], y_base[train_idx]

    estimator_ = clone(self.estimator)
    estimator_.fit(X_train, y_train, **fit_kws)

    self.logger.info("... log things ...")
    self.estimators_.append(estimator_)
    self.split_scores_.append(estimator_.score(X_test, y_test))            
    self.holdout_scores_.append(
        estimator_.score(X_holdout, y_holdout))

In [6]:
from dask_ml.model_selection import train_test_split

class CustomCrossVal(BaseEstimator):
    def __init__(self, client, estimator, cv, logger):
        self.estimator = estimator
        self.cv = cv
        self.logger = logger
        self.client = client
    

    def fit(self, X, y, **fit_kws):



        X_base, X_holdout, y_base, y_holdout = train_test_split(X, y, 
                                                                                                                                    random_state=config.rand_state)

        
        self.split_scores_ = []
        self.holdout_scores_ = []
        self.estimators_ = []            
        

        result_tain_list = []
        result_test_list = []


        param = {"verbosity": 0, "tree_method": "hist", "objective": "reg:squarederror",
                                "max_depth": "10", "learning_rate": 0.01, "max_bin": 256 }


        for train_idx, test_idx in cv.split(X_base, y_base):
            X_train, y_train = X_base[train_idx], y_base[train_idx]
            X_test, y_test = X_base[test_idx], y_base[test_idx]
            dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)
            dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)

            best_model = xgb.dask.train(
                client,
                param, 
                dtrain,
                num_boost_round=500,
                # early_stopping_rounds=50, 
                evals=[(dtrain, "train")],
            )
            
            # booster = output["booster"]
            # print(booster.best_iteration)
            # best_model = booster[: booster.best_iteration]
            train_preds = xgb.dask.predict(client, best_model, dtrain)
            test_preds = xgb.dask.predict(client, best_model, dtest)

            result_tain_list.append(mean_squared_error(train_preds, y_train))
            result_test_list.append(mean_squared_error(test_preds, y_test))




        for train_idx, test_idx in self.cv.split(X_base, y_base):
            

            # print(test_idx)

            X_test, y_test = X_base[test_idx], y_base[test_idx]
            X_train, y_train = X_base.loc[X_base.index[train_idx]], y_base.loc[y_base.index[train_idx]]
        

            # X_test, y_test = X_base[test_idx], y_base[test_idx]
            # X_train, y_train = X_base[train_idx], y_base[train_idx]

            estimator_ = clone(self.estimator)
            estimator_.fit(X_train, y_train, **fit_kws)

            self.logger.info("... log things ...")
            self.estimators_.append(estimator_)
            self.split_scores_.append(estimator_.score(X_test, y_test))            
            self.holdout_scores_.append(
                estimator_.score(X_holdout, y_holdout))
    
        self.best_estimator_ = \
                self.estimators_[np.argmax(self.holdout_scores_)]



        holdout_pred = xgb.dask.predict(client, best_model, X_holdout)

        train_test_diff = np.abs(np.mean(result_tain_list) - np.mean(result_test_list))       

        print('MEAN TRAIN.....', np.mean(result_tain_list))
        print('MEAN TEST.....', np.mean(result_test_list))
        print('TRAIN-TEST DIFF.....', train_test_diff)
        print('HOLDOUT ..... ', mean_squared_error(holdout_pred, y_holdout))                
        return self

    

In [14]:
import xgboost as xgb
xgb.__version__

'0.90'